![](https://miro.medium.com/max/584/1*2lSjt9YKJn9sxK7DSeGDyw.jpeg)

In [3]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [4]:
# Load the data
train = pd.read_csv("../input/digit-recognizer/train.csv")
test = pd.read_csv("../input/digit-recognizer/test.csv")

In [5]:

train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:

test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
x_train = train.drop(labels = ["label"],axis = 1) 
y_train = train["label"]

In [8]:

# Normalize data set to 0-to-1 range
x_train = x_train.astype('float32')
test = test.astype('float32')
x_train /= 255
test /= 255

In [9]:
y_train = keras.utils.to_categorical(y_train, 10)


In [10]:
### Reshaping
x_train = x_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

In [12]:
### Splitting into training and validation set
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2, random_state=42)

In [13]:
### Designing the model
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=(28,28,1), activation="relu"))
model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation="relu"))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation="softmax"))


In [14]:
model.compile(
    loss='categorical_crossentropy',
    optimizer="adam",
    metrics=['accuracy']
)

In [15]:
model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=30,
    validation_data=(x_val, y_val),
    shuffle=True
)


Epoch 1/30
1050/1050 [==============================] - 66s 63ms/step - loss: 0.2012 - accuracy: 0.9343 - val_loss: 0.0599 - val_accuracy: 0.9808
Epoch 2/30
1050/1050 [==============================] - 67s 63ms/step - loss: 0.0698 - accuracy: 0.9785 - val_loss: 0.0376 - val_accuracy: 0.9876
Epoch 3/30
1050/1050 [==============================] - 69s 65ms/step - loss: 0.0539 - accuracy: 0.9837 - val_loss: 0.0339 - val_accuracy: 0.9888
Epoch 4/30
1050/1050 [==============================] - 66s 63ms/step - loss: 0.0455 - accuracy: 0.9860 - val_loss: 0.0303 - val_accuracy: 0.9898
Epoch 5/30
1050/1050 [==============================] - 67s 64ms/step - loss: 0.0395 - accuracy: 0.9877 - val_loss: 0.0289 - val_accuracy: 0.9911
Epoch 6/30
1050/1050 [==============================] - 66s 63ms/step - loss: 0.0341 - accuracy: 0.9894 - val_loss: 0.0243 - val_accuracy: 0.9917
Epoch 7/30
1050/1050 [==============================] - 67s 64ms/step - loss: 0.0315 - accuracy: 0.9903 - val_loss: 0.0257 -

In [16]:
results = model.predict(test)


In [17]:
results = np.argmax(results,axis = 1)
results = pd.Series(results,name="Label")

In [18]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("submission.csv",index=False)

In [19]:
submission

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9
